In [1]:
import pandas as pd
import flair
from textblob import TextBlob
import os
import datetime
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
fmt = '%m/%d/%Y'

In [3]:
def add_to_dict(final_dict, input_dict):
    for item in final_dict:
        input_dict_val = input_dict.get(item, 0)
        final_dict[item] += input_dict_val

In [4]:
def devide_dict_by_scaler(in_dict, val):
    for item in in_dict:
        in_dict[item] /= val

In [5]:
def get_sentiment_report(input_filename, output_filename, start_date='08/01/2021', simulate=False):
    data_df = pd.read_csv(input_filename, index_col='date')
    col = data_df.columns

    temp_c = 0
    for row_i, row in data_df.iterrows():
        temp_c += 1

        if start_date is not None:
            start_date_time_obj = datetime.datetime.strptime(start_date, fmt)
            print(row_i)
            current_date_time_obj = datetime.datetime.strptime(str(row_i), fmt)

            if current_date_time_obj < start_date_time_obj:
                print('Skipping record of date ', str(current_date_time_obj), ' But looking for ',
                      str(start_date_time_obj))
                continue

        total_sentiment_data_count = 0
        pos_dict = dict()
        neg_dict = dict()
        
        pos_total = 0
        neg_total = 0
        
        for col_i in range(len(col)):
            data = (str(row[col_i]))
            # print('\t', col_i)
            if data == 'NaN':
                continue

            if simulate:
                total_sentiment_data_count = 9
            else:
                ss = sid.polarity_scores(data)
                pos_total += ss['pos']
                neg_total += ss['neg']

                total_sentiment_data_count += 1

        print(str(row_i), ' ', temp_c)
       

        pos_dict[str(row_i)] = pos_total / total_sentiment_data_count
        neg_dict[str(row_i)] = neg_total / total_sentiment_data_count
        

        sid_pos_df = pd.DataFrame.from_dict(sid_pos_dict, orient='index',
                                            columns=['gnews_positifitas'])
        sid_pos_df.index.name = 'timestamp'

        sid_neg_df = pd.DataFrame.from_dict(sid_neg_dict, orient='index',
                                            columns=['gnews_negatifitas'])
        sid_neg_df.index.name = 'timestamp'

        final_senti_df = pd.concat([pos_df, neg_df], axis=1)

        if os.path.exists(output_filename):
            keep_header = False
        else:
            keep_header = True

        final_senti_df.to_csv(output_filename, mode='a', header=keep_header)

    return


In [ ]:
if __name__ == "__main__":
    input_filename = 'gnews_data.csv'
    output_filename = input_filename[0:-4] + '_sentiment.csv'
    get_sentiment_report(input_filename, output_filename)